In [ ]:
import biorbd_casadi as biorbd
from bioptim import *
# from ipyvtklink import ViewInteractiveWidget


model_path = "models/pendulum.bioMod"
model = BiorbdModel(model_path)
nq = model.nb_q 

final_time = 1
n_shoot = 30

dynamics = Dynamics(DynamicsFcn.TORQUE_DRIVEN)

x_bounds = model.bounds_from_ranges(["q", "qdot"])  # Use the state bounds defined in the bioMod file
x_bounds[:, [0, -1]] = 0  # Make sure initial position and rotation and velocities (:) are null at the beginning (0) and the end (-1)
x_bounds[1, -1] = 3.14  # Except for the final (-1) rotation (1)

u_bounds = Bounds([-100] * nq, [100] * nq)  # Define all the control bounds to be plus or minus 100
u_bounds[1, :] = 0  # Except for the rotation motor which is deactivated

objective_functions = Objective(ObjectiveFcn.Lagrange.MINIMIZE_CONTROL, key='tau')  # Minimize the generalized forces

x_init = InitialGuess([0] * nq * 2)  # Set initial guesses for generalized coordinates and velocities to 0
u_init = InitialGuess([0] * nq)  # Set initial guesses for generalized forces 0

ocp = OptimalControlProgram(
    model, 
    dynamics, 
    n_shoot, 
    final_time, 
    x_init=x_init, 
    u_init=u_init, 
    x_bounds=x_bounds,
    u_bounds=u_bounds,
    objective_functions=objective_functions,
)


solver = Solver.IPOPT(show_online_optim=False, show_options=dict(print_level=-1))
sol = ocp.solve(solver)
viz = sol.animate(show_now=False)
ViewInteractiveWidget(viz[0].vtk_window.avatar_widget.GetRenderWindow())